In [1]:
import pandas as pd
import numpy as np
import re

# from gensim.models import word2vec

# import matplotlib.pyplot as plt
# import os
import re
import shutil
import string

import pickle

import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# from gensim.models import word2vec

from collections import Counter

# from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

In [2]:
df_all = pd.read_csv('job7_df_all.csv', encoding='utf-8-sig')
# df = pd.read_csv('job7_keyword.csv', encoding='utf-8-sig')
# model = word2vec.Word2Vec.load('word2vec_new.model')

In [3]:
# c0 - c57, 工作內容提取最大65個字 轉成BoW
df_all.iloc[:,705:-1]

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c55,c56,c57,c58,c59,c60,c61,c62,c63,c64
0,149,465,29,1,64,84,354,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,133,244,119,462,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,88,29,395,206,187,418,248,226,5,439,...,0,0,0,0,0,0,0,0,0,0
3,149,353,88,476,36,248,110,5,106,0,...,0,0,0,0,0,0,0,0,0,0
4,149,465,409,110,476,374,248,110,5,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24091,159,324,382,148,231,496,437,519,0,0,...,0,0,0,0,0,0,0,0,0,0
24092,159,324,382,108,231,437,519,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24093,440,454,551,263,423,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24094,159,324,498,577,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X_train, X_valid, y_train, y_valid = train_test_split(df_all.iloc[:,705:-1],\
                                                      df_all['salary_avg'].tolist(),\
                                                      test_size=0.1,  
                                                      random_state=0)

In [5]:
print('Train data len:'+str(len(X_train)))
# print('Class distribution'+str(Counter(y_train)))
print('Valid data len:'+str(len(X_valid)))
# print('Class distribution'+ str(Counter(y_valid)))


Train data len:21686
Valid data len:2410


In [6]:
print('x_train','\n',X_train, end="\n")
print('x_valid','\n',X_valid, end="\n")

print('x_train shape','\n',X_train.shape, end="\n")
print('x_valid shape','\n',X_valid.shape, end="\n")

x_train 
         c0   c1   c2   c3   c4   c5   c6   c7   c8   c9  ...  c55  c56  c57  \
15280  149  465  409   87  327  193  565  546  550  248  ...    0    0    0   
5589   149  465  110   64  212    0    0    0    0    0  ...    0    0    0   
19310  149  300   29  440  395   62  457  479   64  333  ...    0    0    0   
8949    88    1    0    0    0    0    0    0    0    0  ...    0    0    0   
8462    88  317  280  480  360  226  280    0    0    0  ...    0    0    0   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
13123  149   29   88  439   20  248  110   34    5   20  ...    0    0    0   
19648  149  465  300   29  440  382   62  457  178  298  ...    0    0    0   
9845    88  465   29   15  110  508   20  374   32   32  ...    0    0    0   
10799  149   88  206  409  360    0    0    0    0    0  ...    0    0    0   
2732    88  439  387  354  145  480    0    0    0    0  ...    0    0    0   

       c58  c59  c60  c61  c62  c63  c64 

In [7]:
X_train = X_train.values.tolist()
X_valid = X_valid.values.tolist()

In [17]:
with open('vocab.pickle', 'rb') as f:
    vocab = pickle.load(f)

In [19]:
len(vocab)

613

In [8]:
with open('embedding_matrix5.pkl', 'rb') as f:
    embedding_matrix = pickle.load(f)

In [23]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 6.00631571, -3.11916113, -4.56216669, 15.06453228, -2.83644104],
       [-1.33523071, -0.1969057 , -1.10164273,  7.44642019,  5.04957056],
       ...,
       [-2.4779253 , -3.69238138, -3.50253606, 12.97125435, -0.24131088],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

In [11]:

embedding_matrix.shape
embed_dim = 5
max_train_sentence_length = 65
len(X_train[0])

65

In [20]:
len(vocab)

613

In [21]:
#構建TextCNN模型
def TextCNN_model_wv(X_train,y_train,X_test,y_test,embedding_matrix):
    # 模型結構：詞嵌入-卷積池化*3-拼接-全連接-dropout-全連接
    main_input = tf.keras.layers.Input(shape=(max_train_sentence_length,), dtype='float64')
    # 詞嵌入（使用預訓練的詞向量）
    embedder = tf.keras.layers.Embedding(len(vocab) + 1, embed_dim, input_length=max_train_sentence_length, weights=[embedding_matrix], trainable=False)
    #embedder = Embedding(len(vocab) + 1, 300, input_length=50, trainable=False)
    embed = embedder(main_input)

    # 詞窗大小分別爲3,4,5

    ## conv aize = 3
    cnn1 = tf.keras.layers.Conv1D(256, 3, padding='same', strides=1, 
                                    activation='relu',\
                                    kernel_initializer='he_normal',\
                                    kernel_regularizer = regularizers.l2(0.0004))(embed)

    cnn1 = tf.keras.layers.Conv1D(256, 3, padding='same', strides=1, 
                                activation='relu',\
                                kernel_initializer='he_normal',\
                                kernel_regularizer = regularizers.l2(0.0004))(cnn1)
    #max pooling                         
    # cnn1 = tf.keras.layers.MaxPooling1D(pool_size=3, padding='valid')(cnn1)

    cnn1 = tf.keras.layers.Conv1D(128, 3, padding='same', strides=1, 
                                    activation='relu',\
                                    kernel_initializer='he_normal',\
                                    kernel_regularizer = regularizers.l2(0.0004))(cnn1)

    cnn1 = tf.keras.layers.Conv1D(128, 3, padding='same', strides=1, 
                                activation='relu',\
                                kernel_initializer='he_normal',\
                                kernel_regularizer = regularizers.l2(0.0004))(cnn1)
    #max pooling                         
    # cnn1 = tf.keras.layers.MaxPooling1D(pool_size=3, padding='valid')(cnn1)

    ############################################

    ## conv aize = 4
    cnn2 = tf.keras.layers.Conv1D(256, 4, padding='same', strides=1, 
                                    activation='relu',\
                                    kernel_initializer='he_normal',\
                                    kernel_regularizer = regularizers.l2(0.0004))(embed)

    cnn2 = tf.keras.layers.Conv1D(256, 4, padding='same', strides=1, 
                                activation='relu',\
                                kernel_initializer='he_normal',\
                                kernel_regularizer = regularizers.l2(0.0004))(cnn2)
    #max pooling                         
    # cnn2 = tf.keras.layers.MaxPooling1D(pool_size=3, padding='valid')(cnn2)

    cnn2 = tf.keras.layers.Conv1D(128, 4, padding='same', strides=1, 
                                    activation='relu',\
                                    kernel_initializer='he_normal',\
                                    kernel_regularizer = regularizers.l2(0.0004))(cnn2)

    cnn2 = tf.keras.layers.Conv1D(128, 4, padding='same', strides=1, 
                                activation='relu',\
                                kernel_initializer='he_normal',\
                                kernel_regularizer = regularizers.l2(0.0004))(cnn2)
    #max pooling                         
    # cnn2 = tf.keras.layers.MaxPooling1D(pool_size=3, padding='valid')(cnn2)

    ############################################


    ## conv aize = 5
    cnn3 = tf.keras.layers.Conv1D(256, 5, padding='same', strides=1, 
                                    activation='relu',\
                                    kernel_initializer='he_normal',\
                                    kernel_regularizer = regularizers.l2(0.0004))(embed)

    cnn3 = tf.keras.layers.Conv1D(256, 5, padding='same', strides=1, 
                                activation='relu',\
                                kernel_initializer='he_normal',\
                                kernel_regularizer = regularizers.l2(0.0004))(cnn3)
    #max pooling                         
    # cnn3 = tf.keras.layers.MaxPooling1D(pool_size=3, padding='valid')(cnn3)

    cnn3 = tf.keras.layers.Conv1D(128, 5, padding='same', strides=1, 
                                    activation='relu',\
                                    kernel_initializer='he_normal',\
                                    kernel_regularizer = regularizers.l2(0.0004))(cnn3)

    cnn3 = tf.keras.layers.Conv1D(128, 5, padding='same', strides=1, 
                                activation='relu',\
                                kernel_initializer='he_normal',\
                                kernel_regularizer = regularizers.l2(0.0004))(cnn3)
    #max pooling                         
    # cnn3 = tf.keras.layers.MaxPooling1D(pool_size=3, padding='valid')(cnn3)

    ############################################


    # 合併三個模型的輸出向量
    cnn = tf.keras.layers.concatenate([cnn1, cnn2, cnn3], axis=-1)
    flat = tf.keras.layers.Flatten()(cnn)

    # drop1 = tf.keras.layers.Dropout(0.5)(flat)

    dense = tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal',kernel_regularizer=regularizers.L1(0.1))(flat)
    
    # drop1 = tf.keras.layers.Dropout(0.5)(dense)

    dense = tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal',kernel_regularizer=regularizers.L1(0.1))(dense)

    dense = tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal',kernel_regularizer=regularizers.L1(0.05))(dense)

    # drop1 = tf.keras.layers.Dropout(0.5)(dense)

    dense = tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal',kernel_regularizer=regularizers.L1(0.05))(dense)

    # drop1 = tf.keras.layers.Dropout(0.5)(dense)
    dense = tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal',kernel_regularizer=regularizers.L1(0.05))(dense)

    dense = tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal')(dense)

    main_output = tf.keras.layers.Dense(1, activation='linear')(dense)


    # model main_output 
    ##########################
    batch_size=32 
    epochs=50
    
    ##########################
    
    train_ds = tf.data.Dataset.from_tensor_slices((X_train,y_train))
    valid_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test))


    model = tf.keras.Model(inputs=main_input, outputs=main_output)

    model.compile(loss='mse', optimizer=tf.keras.optimizers.experimental.AdamW(learning_rate=0.004), 
                    metrics=[tf.keras.metrics.MeanSquaredError()])

    print(model.summary())
    
    # model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs)

    history = model.fit(train_ds.shuffle(2000).batch(batch_size),
                    epochs= epochs,
                    validation_data=valid_ds.batch(batch_size),
                    verbose=1)


    # output result and plot loss trending
    # result = model.predict(X_valid)
    # print('MSE: ', tf.keras.metrics.MeanSquaredError(y_valid, result))

    plt.plot(history.history[ 'loss' ])
    plt.plot(history.history[ 'val_loss' ])
    plt.title( 'model loss' )
    plt.ylabel( 'loss' )
    plt.xlabel( 'epoch' )
    plt.legend([ 'train' , 'valid' ], loc= 'lower right' )
    plt.show()



In [24]:
y_train = np.asarray(y_train)
y_valid = np.asarray(y_valid)

TextCNN_model_wv(X_train,y_train,X_valid,y_valid,embedding_matrix)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 65)]         0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 65, 5)        3070        ['input_3[0][0]']                
                                                                                                  
 conv1d_12 (Conv1D)             (None, 65, 256)      4096        ['embedding_2[0][0]']            
                                                                                                  
 conv1d_16 (Conv1D)             (None, 65, 256)      5376        ['embedding_2[0][0]']            
                                                                                            

KeyboardInterrupt: 